# Análise de Jogos da NFL com Regressão Logística

Este notebook utiliza a regressão logística para prever o vencedor de jogos da NFL com base em vitórias e derrotas acumuladas dos times. A seguir, apresentamos o passo a passo do código:

---

## 1. Importação de Bibliotecas

Importamos as bibliotecas necessárias para manipulação de dados, divisão de conjuntos, criação do modelo e avaliação de desempenho.

## 2. Carregamento do dataset

Utilizei o datset "games.csv", que contém dados de jogos da NFL de 1999 até os jogos atuais, o dataset é atualizado toda semana e eu atualizo através desse repositório: "https://github.com/nflverse/nfldata/blob/master/data/games.csv"

## 3. Filtragem dos dados
Filtrei apenas os jogos de temporada regular, de 2013 pra frente

## 4. Criação de features

Vitória e derrotas acumuladas no time de casa e fora

## 5. Divisão em treino e teste

Utilizei 80% dos dados para treino e 20% para teste

## 6. Treinamento do modelo

Criei um moelo de regressão logística, que visa predizer o vencedor da partida

## 7. Avaliação do modelo

Calcula a acurácia do modelo

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from rich.console import Console
from rich.table import Table

In [2]:
# Carregar dataset
df = pd.read_csv('games.csv')

# Filtrar temporadas a partir de 2013 e jogos da temporada regular
df = df[df['season'] >= 2013]
df = df[df['game_type'] == 'REG']

# Definir o vencedor (1 se o time da casa vencer, 0 se o time visitante vencer)
df['winner'] = df['result'].apply(lambda x: 1 if x > 0 else 0)
df = df[df['result'] != 0]

# Contar vitórias do time da casa
df['home_team_wins'] = df.groupby(['home_team', 'season'])['winner'].cumsum()

# Contar vitórias do time visitante (inverso das vitórias do time da casa)
df['away_team_wins'] = df.groupby(['away_team', 'season'])['winner'].transform(lambda x: (1 - x).cumsum())

# Contar derrotas do time da casa (inverso das vitórias do time da casa)
df['home_team_losses'] = df.groupby(['home_team', 'season'])['winner'].transform(lambda x: (1 - x).cumsum())

# Contar derrotas do time visitante
df['away_team_losses'] = df.groupby(['away_team', 'season'])['winner'].cumsum()

# Selecionar as colunas de interesse para o modelo
parametros = ['home_team_wins', 'away_team_wins', 'home_team_losses', 'away_team_losses']

# Dividir em treino e teste
df_train = df[df['season'] < 2023]
df_test = df[df['season'] >= 2023]

X_train = df_train[parametros]
y_train = df_train['winner']
X_test = df_test[parametros]
y_test = df_test['winner']

# Criar o modelo de regressão logística
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# Fazer previsões
y_pred = modelo.predict(X_test)

# Avaliar a acurácia do modelo
acuracia = accuracy_score(y_test, y_pred)
print(f'Acurácia: {acuracia * 100:.2f}%')


Acurácia: 84.38%


In [3]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(modelo, X_train, y_train, cv=5)  # 5-fold cross-validation
print(f'Acurácia média com cross-validation: {scores.mean() * 100:.2f}%')


Acurácia média com cross-validation: 84.38%


In [4]:
# Instanciar o console do Rich
console = Console()

# Variável para o número da semana que você deseja visualizar
semana = int(input("Digite o número da semana (por exemplo, 1 para semana 1): "))

# Filtrar os jogos da temporada de 2024 para a semana desejada
df_semana = df_test[(df_test['season'] == 2024) & (df_test['week'] == semana)].copy()

# Fazer previsões de probabilidade
y_proba_semana = modelo.predict_proba(df_semana[parametros])

# Adicionar as colunas de probabilidade no DataFrame
df_semana['home_team_win_prob'] = y_proba_semana[:, 1]  # Probabilidade do time da casa vencer
df_semana['away_team_win_prob'] = y_proba_semana[:, 0]  # Probabilidade do time visitante vencer

# Adicionar a coluna de vencedor previsto
df_semana['predicted_winner'] = modelo.predict(df_semana[parametros])

# Salvar as previsões no arquivo CSV
arquivo_csv = f"Previsões_semana_{semana}.csv"
df_semana.to_csv(arquivo_csv, index=False)
console.print(f"[bold green]As previsões foram salvas no arquivo '{arquivo_csv}'.[/bold green]")

# Criar a tabela do Rich
table = Table(title=f"Predições para os Jogos da Semana {semana}")

# Adicionar colunas na tabela
table.add_column("Time da Casa", style="bold cyan")
table.add_column("Time Visitante", style="bold cyan")
table.add_column("Chance Casa (%)", style="green")
table.add_column("Chance Visitante (%)", style="red")
table.add_column("Vencedor Previsto", style="bold magenta")

# Adicionar as linhas de predições
for _, row in df_semana.iterrows():
    vencedor = row['home_team'] if row['predicted_winner'] == 1 else row['away_team']
    table.add_row(
        row['home_team'],
        row['away_team'],
        f"{row['home_team_win_prob']:.2%}",
        f"{row['away_team_win_prob']:.2%}",
        vencedor
    )

# Exibir a tabela
console.print(table)

Digite o número da semana (por exemplo, 1 para semana 1):  1


As previsões foram salvas no arquivo 'Previsões_semana_1.csv'.

                             Predições para os Jogos da Semana 1                              
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Time da Casa ┃ Time Visitante ┃ Chance Casa (%) ┃ Chance Visitante (%) ┃ Vencedor Previsto ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ KC           │ BAL            │ 81.14%          │ 18.86%               │ KC                │
│ PHI          │ GB             │ 81.14%          │ 18.86%               │ PHI               │
│ ATL          │ PIT            │ 22.83%          │ 77.17%               │ PIT               │
│ BUF          │ ARI            │ 81.14%          │ 18.86%               │ BUF               │
│ CHI          │ TEN            │ 81.14%          │ 18.86%               │ CHI               │
│ CIN          │ NE             │ 22.83%          │ 77.17%               │ NE                │
│ IND          │ HOU            │ 22.83%          │ 77.17%               │ HOU               │
│ MIA          │ JAX            │ 81.14%          │ 18.86%               │ MIA               │
│ NO           │ CAR            │ 81.14%          │ 18.86%               │ NO                │
│ NYG          │ MIN            │ 22.83%          │ 77.17%               │ MIN               │
│ LAC          │ LV             │ 81.14%          │ 18.86%               │ LAC               │
│ SEA          │ DEN            │ 81.14%          │ 18.86%               │ SEA               │
│ CLE          │ DAL            │ 22.83%          │ 77.17%               │ DAL               │
│ TB           │ WAS            │ 81.14%          │ 18.86%               │ TB                │
│ DET          │ LA             │ 81.14%          │ 18.86%               │ DET               │
│ SF           │ NYJ            │ 81.14%          │ 18.86%               │ SF                │
└──────────────┴────────────────┴─────────────────┴──────────────────────┴───────────────────┘

In [5]:
def calcular_lucro(row, valor_entrada):
    if row['predicted_winner'] == 1:  # Aposta no time da casa
        odds = row['home_moneyline']
        if row['winner'] == 1:  # Aposta certa (time da casa venceu)
            return valor_entrada * (odds / 100) if odds > 0 else valor_entrada * (100 / abs(odds))
        else:  # Aposta errada (time da casa perdeu)
            return -valor_entrada
    elif row['predicted_winner'] == 0:  # Aposta no time visitante
        odds = row['away_moneyline']
        if row['winner'] == 0:  # Aposta certa (time visitante venceu)
            return valor_entrada * (odds / 100) if odds > 0 else valor_entrada * (100 / abs(odds))
        else:  # Aposta errada (time visitante perdeu)
            return -valor_entrada
    else:  # Caso não haja previsão
        return 0

In [6]:
def moneyline_to_decimal(moneyline):
    if moneyline > 0:
        return (moneyline / 100) + 1
    elif moneyline < 0:
        return (100 / abs(moneyline)) + 1
    else:
        return None  # Odds inválidas

In [7]:
from rich.console import Console
from rich.table import Table# Instanciar o console do Rich
console = Console()

# Variável para o número da semana que você deseja visualizar
semana = int(input("Digite o número da semana (por exemplo, 1 para semana 1): "))

# Perguntar o valor da entrada nos jogos
valor_entrada = float(input("Digite o valor da entrada para cada jogo (exemplo: 100): "))

# Carregar as previsões para a semana escolhida
previsao = pd.read_csv(f'Previsões_semana_{semana}.csv')

# Filtrar os jogos da temporada de 2024 para a semana desejada
df_semana = df_test[(df_test['season'] == 2024) & (df_test['week'] == semana)].copy()

# Converter odds Moneyline para Decimais
df_semana['home_odds_decimal'] = df_semana['home_moneyline'].apply(moneyline_to_decimal)
df_semana['away_odds_decimal'] = df_semana['away_moneyline'].apply(moneyline_to_decimal)

# Adicionar coluna de previsão de vencedor
if 'predicted_winner' not in df_semana.columns:
    df_semana['predicted_winner'] = "Sem previsão"  # Inicializar a coluna, se não existir
else:
    df_semana['predicted_winner'] = df_semana['predicted_winner'].fillna("Sem previsão")

# Atribuir valores da previsão de acordo com o CSV
df_semana['predicted_winner'] = df_semana['game_id'].map(
    previsao.set_index('game_id')['predicted_winner']
)

# Calcular o lucro para cada jogo
df_semana['lucro'] = df_semana.apply(calcular_lucro, axis=1, valor_entrada=valor_entrada)

# Criar a tabela do Rich
table = Table(title=f"Comparação de Previsões para os Jogos da Semana {semana}")

# Adicionar colunas na tabela
table.add_column("Time da Casa", style="bold cyan")
table.add_column("Time Visitante", style="bold cyan")
table.add_column("Vencedor Previsto", style="bold magenta")
table.add_column("Vencedor Real", style="yellow")
table.add_column("Odds Casa", style="green")
table.add_column("Odds Visitante", style="red")
table.add_column("Lucro", style="bold green")

# Adicionar as linhas com os dados para cada jogo
for _, row in df_semana.iterrows():
    predicted_team = row['home_team'] if row['predicted_winner'] == 1 else (
        row['away_team'] if row['predicted_winner'] == 0 else "Sem previsão"
    )
    real_winner_team = (
        row['home_team'] if row['winner'] == 1 
        else row['away_team'] if row['winner'] == 0 
        else "Ainda não definido"
    )
    lucro = row['lucro'] if 'lucro' in row else 0
    
    table.add_row(
        row['home_team'],
        row['away_team'],
        predicted_team,
        real_winner_team,
        f"{row['home_odds_decimal']:.2f}",
        f"{row['away_odds_decimal']:.2f}",
        f"${lucro:.2f}"
    )

# Exibir a tabela
console.print(table)

# Calcular métricas da semana
acertos = (df_semana['predicted_winner'] == df_semana['winner']).sum()
total_jogos = len(df_semana[df_semana['winner'].notna()])  # Ignorar jogos sem resultado
assertividade = (acertos / total_jogos * 100) if total_jogos > 0 else 0
lucro_total = df_semana['lucro'].sum() if 'lucro' in df_semana else 0
porcentagem_lucro = (lucro_total / (total_jogos * valor_entrada)) * 100 if total_jogos > 0 else 0

# Exibir resultado geral da semana
result_table = Table(title="Resultados da Semana")
result_table.add_column("Métrica", style="bold cyan")
result_table.add_column("Valor", style="bold magenta")

result_table.add_row("Total de jogos", str(total_jogos))
result_table.add_row("Acertos", str(acertos))
result_table.add_row("Assertividade", f"{assertividade:.2f}%")
result_table.add_row("Lucro total", f"${lucro_total:.2f}")
result_table.add_row("Porcentagem de lucro", f"{porcentagem_lucro:.2f}%")

console.print(result_table)

Digite o número da semana (por exemplo, 1 para semana 1):  15
Digite o valor da entrada para cada jogo (exemplo: 100):  5


                            Comparação de Previsões para os Jogos da Semana 15                             
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Time da Casa ┃ Time Visitante ┃ Vencedor Previsto ┃ Vencedor Real ┃ Odds Casa ┃ Odds Visitante ┃ Lucro  ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ SF           │ LA             │ LA                │ LA            │ 1.65      │ 2.30           │ $6.50  │
│ CAR          │ DAL            │ DAL               │ DAL           │ 1.70      │ 2.20           │ $6.00  │
│ CLE          │ KC             │ KC                │ KC            │ 2.90      │ 1.43           │ $2.17  │
│ HOU          │ MIA            │ HOU               │ HOU           │ 1.65      │ 2.30           │ $3.23  │
│ JAX          │ NYJ            │ NYJ               │ NYJ           │ 2.45      │ 1.57           │ $2.86  │
│ NO           │ WAS            │ WAS               │ WAS           │ 3.95      │ 1.27           │ $1.33  │
│ NYG          │ BAL            │ BAL               │ BAL           │ 11.00     │ 1.06           │ $0.28  │
│ TEN          │ CIN            │ CIN               │ CIN           │ 3.05      │ 1.40           │ $2.00  │
│ ARI          │ NE             │ ARI               │ ARI           │ 1.39      │ 3.10           │ $1.94  │
│ DEN          │ IND            │ DEN               │ DEN           │ 1.42      │ 2.95           │ $2.10  │
│ DET          │ BUF            │ DET               │ BUF           │ 1.77      │ 2.10           │ $-5.00 │
│ LAC          │ TB             │ TB                │ TB            │ 1.62      │ 2.36           │ $6.80  │
│ PHI          │ PIT            │ PIT               │ PHI           │ 1.38      │ 3.15           │ $-5.00 │
│ SEA          │ GB             │ GB                │ GB            │ 2.36      │ 1.62           │ $3.09  │
│ MIN          │ CHI            │ MIN               │ CHI           │ 1.33      │ 3.45           │ $-5.00 │
│ LV           │ ATL            │ ATL               │ ATL           │ 3.20      │ 1.37           │ $1.85  │
└──────────────┴────────────────┴───────────────────┴───────────────┴───────────┴────────────────┴────────┘

      Resultados da Semana       
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Métrica              ┃ Valor  ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Total de jogos       │ 16     │
│ Acertos              │ 13     │
│ Assertividade        │ 81.25% │
│ Lucro total          │ $25.15 │
│ Porcentagem de lucro │ 31.43% │
└──────────────────────┴────────┘